# Streaming Data to Plotly

If you are reading data for a long time for an experiment that runs for a few hours or days, then you may wish to monitor your data online from a remote location.

To accomplish such a task, you will have to periodically read the data and then stream the data to a web server. The `plotly` package is the perfect tool for this task. 

In a later tutorial, I discuss reading data from an Arduino using `pyserial`. Presumably this data could then be streamed. However, in this tutorial I will simply calculate a quantity (instead of reading a value from a sensor) and stream it. 

## Plotly Account and Stream Tokens

Streaming data requires having an account on Plotly and requires creating a stream token in your account. See this [tutorial](https://plot.ly/python/streaming-tutorial/) for setting up your ``.credentials`` file that contains your username, key, and stream tokens. Each data stream should have its own stream token. This is the unique identifier that connects your Python program to a particular graph  on the plot.ly site. Stream tokens are called `stream_ids` in your credentials file.

It is possible to enter credentials (including stream tokens) as a string in your code; however, if you share your code, then you will allow others to stream data to your account. Therefore, to maintain security, I recommend storing your username, key, and stream_ids in a ``.credentials`` file on your local computer.

## Python Program

After obtaining a stream token and creating your credentials file according to the tutorial at plot.ly, continue writing your Python program.

First, import packages.

In [ ]:
import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go
import numpy as np
import time

Get the stream token.

In [7]:
# Get stream id from stream id list 
stream_ids = tls.get_credentials_file()['stream_ids']
stream_id = stream_ids[0]

# Make instance of stream id object 
stream_1 = go.Stream(
    token=stream_id  # link stream id to 'token' key
)


Initialize the plotly graph. Set up the trace, label axes, create the title, etc. Then open the connection.

In [11]:
# Initialize trace of streaming plot by embedding the unique stream_id
trace1 = go.Scatter(
    x=[],
    y=[],
    mode='lines+markers',
    stream=stream_1         # (!) embed stream id, 1 per trace
)

data = go.Data([trace1])

# Add title to layout object
layout = go.Layout(title='cos(omega*t)')

# Make a figure object
fig = go.Figure(data=data, layout=layout)

# Send fig to plot.ly, initialize streaming plot
py.iplot(fig, filename='cosine function')

# We will provide the stream link object the same token that's associated with the trace we wish to stream to
s = py.Stream(stream_id)

# We then open a connection
s.open()

If you are reading data from an Arduino or an interface connected to the computer, then you should add a function to read the data. In a later tutorial, I demonstrate how to read data from an Arduino using the `pyserial` package. In this case, we will simply calculate $cos(\omega t)$ and stream the result to the plot.ly site. 

In [ ]:
start=time.time()
omega = np.pi #angular frequency in rad/s

while True:

    # Current time on horizontal axis, cos(omega*t) on the vertical axis
    t = time.time()-start
    f = np.cos(omega*t)
    
    # Send data to your plot
    #     Write numbers to stream to append current data on plot,
    #     write lists to overwrite existing data on plot
    s.write(dict(x=t, y=f))

    time.sleep(0.1)  # sleep in seconds  
    
s.close()